In [12]:
import scanpy as scp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
from datetime import datetime

from auxiliar import *
from scmap import *


import mygene

# Susanne

In [118]:
adata_reference = scp.read("data_other_studies/susanne/adata.h5ad")
m = scp.read_mtx("data_other_studies/susanne/matrix.mtx").T
gs = pd.read_csv("data_other_studies/susanne/genes.tsv",sep="\t",header=None)[0].values
genes = set(adata_reference.var["gene_ids"].values)
keep_genes = [g in genes for g in gs]
cs = pd.read_csv("data_other_studies/susanne/barcodes.tsv",sep="\t",header=None).values[:,0]
cells = set(adata_reference.obs.index)
keep_cells = [i in cells for i in cs]    
adata_reference.layers["counts"] = m[keep_cells, keep_genes].X
adata_reference.layers["counts_log1p"] = np.log1p(adata_reference.layers["counts"])
adata_reference.obs["annotation"] = adata_reference.obs["leiden"].values
adata_reference.var["gene_names"] = adata_reference.var.index.values

adata_reference.write("data_other_studies/susanne/adata_curated.h5ad")

/Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/anndata/compat/__init__.py:363: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/anndata/compat/__init__.py:363: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


# Prisca

In [100]:
adata_reference = scp.read("data_other_studies/prisca/adata.h5ad")
adata_reference.layers["counts_log1p"] = np.log1p(adata_reference.layers["counts"])
adata_reference.obs["annotation"] = adata_reference.obs["celltypeannotation"].values

scp.pp.log1p(adata_reference)
scp.pp.highly_variable_genes(adata_reference, n_top_genes=2000, flavor="seurat")

gene_symbols = list(map(lambda x : x.split(".")[0], adata_reference.var.index.values))
mg = mygene.MyGeneInfo()
# Convert gene symbols to Ensembl IDs
result = mg.querymany(gene_symbols, scopes='symbol', fields='ensembl.gene', species='mouse')

gene_names = [i["query"] for i in result if "query" in i if "ensembl" in i]
d = pd.DataFrame(gene_names)
d["count"] = 1
d = d.groupby(0).count().reset_index()
d = d.sort_values("count", ascending=False, inplace=False)
duplicated = d[d["count"] > 1].iloc[:,0].values

mapping = {
    i["query"]:i["ensembl"]["gene"] if "ensembl" in i and "gene" in i["ensembl"] and i not in duplicated else None for i in result
}

gene_names = adata_reference.var.index.values
gene_ids = [mapping[i] if i in mapping else None for i in adata_reference.var.index.values]

adata_reference.var["gene_ids"] = gene_ids
adata_reference.var["gene_names"] = gene_names

adata_reference = adata_reference[:,-adata_reference.var["gene_ids"].isna()].copy()

adata_reference.write("data_other_studies/prisca/adata_curated.h5ad")


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
2212 input query terms found dup hits:	[('4933401J01Rik', 2), ('E330040D14Rik', 2), ('4921511E07Rik', 2), ('4930486I03Rik', 2), ('Eef1a1-ps
2316 input query terms found no hit:	['AC125120', 'Pih1d3', '1700027A15Rik', 'n-R5s213', 'AC167036', 'AC123856', 'AC147806', 'AC147806', 


# Pijuan-Sala

In [ ]:
adata_reference = scp.read("data_other_studies/pijuan/adata.h5ad")
adata_reference.layers["counts"] = adata_reference.X.copy()
adata_reference.layers["counts_log1p"] = np.log1p(adata_reference.layers["counts"])
adata_reference.obs["annotation"] = adata_reference.obs["celltype"].values
adata_reference.obsm["X_umap"] = adata_reference.obs[["umapX","umapY"]].values
adata_reference.var_names = pd.read_csv("data_other_studies/pijuan/genes.tsv", sep="\t", header=None)[0].values

scp.pp.log1p(adata_reference)
scp.pp.highly_variable_genes(adata_reference, n_top_genes=2000, flavor="seurat")

adata_reference = adata_reference[adata_reference.obs["umapX"].notna(),:]

gene_symbols = list(map(lambda x : x.split(".")[0], adata_reference.var.index.values))
mg = mygene.MyGeneInfo()
# Convert gene symbols to Ensembl IDs
result = mg.querymany(gene_symbols, scopes='ensembl.gene', fields='symbol', species='mouse')

gene_names = [i["query"] for i in result if "query" in i if "symbol" in i]
d = pd.DataFrame(gene_names)
d["count"] = 1
d = d.groupby(0).count().reset_index()
d = d.sort_values("count", ascending=False, inplace=False)
duplicated = d[d["count"] > 1].iloc[:,0].values

mapping = {
    i["query"]:i["symbol"] if "symbol" in i and i not in duplicated else None for i in result
}

gene_ids = adata_reference.var.index.values
gene_names = [mapping[i] if i in mapping else None for i in adata_reference.var.index.values]

adata_reference.var["gene_ids"] = gene_ids
adata_reference.var["gene_names"] = gene_names

adata_reference = adata_reference[:,-adata_reference.var["gene_ids"].isna()].copy()

adata_reference.write("data_other_studies/pijuan/adata_curated.h5ad")

/Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## Save analysis script

In [121]:
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

# Construct the output file name with the current date
input_file = f"setup_external_datasets.ipynb"
output_file = f"script_versions/setup_external_datasets_{current_date}"

# Convert the notebook to HTML with the new file name
!jupyter nbconvert --to html "$input_file" --output "$output_file"

[NbConvertApp] Converting notebook setup_external_datasets.ipynb to html
[NbConvertApp] Writing 305122 bytes to script_versions/setup_external_datasets_20250604.html
